In [1]:
import numpy as np
import math
import time
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,cross_validate,StratifiedKFold,RepeatedStratifiedKFold

from sklearn.preprocessing import MultiLabelBinarizer,MinMaxScaler
import ast

# https://github.com/scikit-learn/scikit-learn/issues/3570
from functools import partial
from sklearn.metrics import confusion_matrix,precision_score, make_scorer,recall_score,accuracy_score,classification_report

In [2]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [ ]:
path = './results/'

In [5]:

# load the results_radius.csv file
df_original = pd.read_csv("results_radius.csv".format(path))

df_original['phi'] = df_original['phi'].apply(ast.literal_eval)
df_original['haralick'] = df_original['haralick'].apply(ast.literal_eval)
df_original['lbp'] = df_original['lbp'].apply(ast.literal_eval)


# get only the values for radius = 2
radius_curr = 2
df = df_original.loc[df_original['radius'] == radius_curr]

In [6]:
# convert the arrays to columns in different dataframes
df2 = pd.DataFrame(df['phi'].values.tolist(), index= df.index)
df3 = pd.DataFrame(df['haralick'].values.tolist(), index= df.index)
df4 = pd.DataFrame(df['lbp'].values.tolist(), index= df.index)

In [7]:
# merge the dataframes
dfFinal = pd.concat([df2, df3, df4], axis=1, ignore_index=True)

# X is the vectors to be used in classification
X = dfFinal
# normalize the values to interval [0, 1]
X = pd.DataFrame(MinMaxScaler().fit_transform(X))

# y is the labels of vectors X
y = df.iloc[:,[-1]]

In [8]:
# define function to iterate and get the metrics for each class
def mean_dict(curIt, curDict, newDict):
  # curDict = (curDict*curIt + newDict) / (curIt+1)
  curDict['consolidacao']['precision'] = (curDict['consolidacao']['precision']*curIt + newDict['consolidacao']['precision'])/(curIt+1)
  curDict['consolidacao']['recall'] = (curDict['consolidacao']['recall']*curIt + newDict['consolidacao']['recall'])/(curIt+1)
  curDict['consolidacao']['f1-score'] = (curDict['consolidacao']['f1-score']*curIt + newDict['consolidacao']['f1-score'])/(curIt+1)
  
  curDict['enfisema']['precision'] = (curDict['enfisema']['precision']*curIt + newDict['enfisema']['precision'])/(curIt+1)
  curDict['enfisema']['recall'] = (curDict['enfisema']['recall']*curIt + newDict['enfisema']['recall'])/(curIt+1)
  curDict['enfisema']['f1-score'] = (curDict['enfisema']['f1-score']*curIt + newDict['enfisema']['f1-score'])/(curIt+1)
  
  curDict['espessamento']['precision'] = (curDict['espessamento']['precision']*curIt + newDict['espessamento']['precision'])/(curIt+1)
  curDict['espessamento']['recall'] = (curDict['espessamento']['recall']*curIt + newDict['espessamento']['recall'])/(curIt+1)
  curDict['espessamento']['f1-score'] = (curDict['espessamento']['f1-score']*curIt + newDict['espessamento']['f1-score'])/(curIt+1)
  
  curDict['favo_de_mel']['precision'] = (curDict['favo_de_mel']['precision']*curIt + newDict['favo_de_mel']['precision'])/(curIt+1)
  curDict['favo_de_mel']['recall'] = (curDict['favo_de_mel']['recall']*curIt + newDict['favo_de_mel']['recall'])/(curIt+1)
  curDict['favo_de_mel']['f1-score'] = (curDict['favo_de_mel']['f1-score']*curIt + newDict['favo_de_mel']['f1-score'])/(curIt+1)
  
  curDict['normal']['precision'] = (curDict['normal']['precision']*curIt + newDict['normal']['precision'])/(curIt+1)
  curDict['normal']['recall'] = (curDict['normal']['recall']*curIt + newDict['normal']['recall'])/(curIt+1)
  curDict['normal']['f1-score'] = (curDict['normal']['f1-score']*curIt + newDict['normal']['f1-score'])/(curIt+1)
  
  curDict['vidro_fosco']['precision'] = (curDict['vidro_fosco']['precision']*curIt + newDict['vidro_fosco']['precision'])/(curIt+1)
  curDict['vidro_fosco']['recall'] = (curDict['vidro_fosco']['recall']*curIt + newDict['vidro_fosco']['recall'])/(curIt+1)
  curDict['vidro_fosco']['f1-score'] = (curDict['vidro_fosco']['f1-score']*curIt + newDict['vidro_fosco']['f1-score'])/(curIt+1)
  
  curDict['macro avg']['precision'] = (curDict['macro avg']['precision']*curIt + newDict['macro avg']['precision'])/(curIt+1)
  curDict['macro avg']['recall'] = (curDict['macro avg']['recall']*curIt + newDict['macro avg']['recall'])/(curIt+1)
  curDict['macro avg']['f1-score'] = (curDict['macro avg']['f1-score']*curIt + newDict['macro avg']['f1-score'])/(curIt+1)
  
  curDict['weighted avg']['precision'] = (curDict['weighted avg']['precision']*curIt + newDict['weighted avg']['precision'])/(curIt+1)
  curDict['weighted avg']['recall'] = (curDict['weighted avg']['recall']*curIt + newDict['weighted avg']['recall'])/(curIt+1)
  curDict['weighted avg']['f1-score'] = (curDict['weighted avg']['f1-score']*curIt + newDict['weighted avg']['f1-score'])/(curIt+1)
  
  curDict['accuracy'] = (curDict['accuracy']*curIt + newDict['accuracy'])/(curIt+1)

In [9]:
best = (0,0)
dict_repeated = {
    'consolidacao': 0.0,
    'enfisema': 0.0,
    'espessamento': 0.0,
    'favo_de_mel': 0.0,
    'normal': 0.0,
    'vidro_fosco': 0.0,
    'accuracy': 0.0,
    'macro avg': 0.0,
    'weighted avg': 0.0
}
dict_repeated = {'consolidacao': {'precision': 0.8318777292576419, 'recall': 0.844789356984479, 'f1-score': 0.8382838283828383, 'support': 451}, 'enfisema': {'precision': 0.8772277227722772, 'recall': 0.8824701195219123, 'f1-score': 0.8798411122144985, 'support': 502}, 'espessamento': {'precision': 0.7375886524822695, 'recall': 0.7050847457627119, 'f1-score': 0.7209705372616984, 'support': 590}, 'favo_de_mel': {'precision': 0.8176691729323309, 'recall': 0.8207547169811321, 'f1-score': 0.8192090395480227, 'support': 530}, 'normal': {'precision': 0.8552631578947368, 'recall': 0.8813559322033898, 'f1-score': 0.8681135225375627, 'support': 590}, 'vidro_fosco': {'precision': 0.8883248730964467, 'recall': 0.8823529411764706, 'f1-score': 0.8853288364249579, 'support': 595}, 'accuracy': 0.8348680171884592, 'macro avg': {'precision': 0.8346585514059505, 'recall': 0.8361346354383494, 'f1-score': 0.8352911460615964, 'support': 3258}, 'weighted avg': {'precision': 0.8340226218526197, 'recall': 0.8348680171884592, 'f1-score': 0.8343331150543905, 'support': 3258}}
for it in range(100):
  print(">>>> it: ", it)
  conf_matrix_list_of_arrays = []
  accuracy_list = []
  recall_list = []
  precision_list = []

  y_true_full = np.array([])
  y_pred_full = np.array([])

	# init KNN classifier with K=5
  classifier_3NN = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
  # init validation method with 10 splits
  skf = StratifiedKFold(n_splits = 10, shuffle=True, random_state=None)

  for train_index, test_index in skf.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # set the training dataset on KNN
    classifier_3NN.fit(X_train,y_train.values.ravel())

    # predict the labels on test set
    y_pred = classifier_3NN.predict(X_test)
    # actual values of the test set
    y_true = y_test.values.ravel()

    # concatenate the results for every individual on a full array
    y_pred_full = np.concatenate((y_pred_full, y_pred))
    y_true_full = np.concatenate((y_true_full, y_true))

    # get the current confusion matrix
    conf_matrix = confusion_matrix(y_true, y_pred)
    
    # get the metrics
    a = accuracy_score(y_true, y_pred)
    r = recall_score(y_true, y_pred, average='macro')
    p = precision_score(y_true, y_pred, average='macro')

    # append the results on a list
    conf_matrix_list_of_arrays.append(conf_matrix)
    accuracy_list.append(a)
    recall_list.append(r)
    precision_list.append(p)

  # sum of all the confusion matrices
  sum_of_conf_matrix_arrays = np.sum(conf_matrix_list_of_arrays, axis=0)

  # get the classification report
  dict_ret = classification_report(y_true_full, y_pred_full, output_dict=True)

  # gets the mean of the confusion matrices using the methos defines above
  mean_dict(it, dict_repeated, dict_ret)
  print(dict_repeated)

print("Final\n",dict_repeated)


>>>> it:  0
{'consolidacao': {'precision': 0.8473118279569892, 'recall': 0.8736141906873615, 'f1-score': 0.8602620087336245, 'support': 451}, 'enfisema': {'precision': 0.9023904382470119, 'recall': 0.9023904382470119, 'f1-score': 0.9023904382470119, 'support': 502}, 'espessamento': {'precision': 0.7767857142857143, 'recall': 0.7372881355932204, 'f1-score': 0.7565217391304349, 'support': 590}, 'favo_de_mel': {'precision': 0.8509433962264151, 'recall': 0.8509433962264151, 'f1-score': 0.8509433962264151, 'support': 530}, 'normal': {'precision': 0.8602329450915142, 'recall': 0.8762711864406779, 'f1-score': 0.8681780016792612, 'support': 590}, 'vidro_fosco': {'precision': 0.8866666666666667, 'recall': 0.8941176470588236, 'f1-score': 0.890376569037657, 'support': 595}, 'accuracy': 0.8538980969920197, 'macro avg': {'precision': 0.8540551647457186, 'recall': 0.8557708323755852, 'f1-score': 0.8547786921757341, 'support': 3258}, 'weighted avg': {'precision': 0.85314466240265, 'recall': 0.8538980

In [10]:
import json
print(json.dumps(dict_repeated,sort_keys=True, indent=4))

{
    "accuracy": 0.8547912829957036,
    "consolidacao": {
        "f1-score": 0.8611973022993691,
        "precision": 0.8516239382612661,
        "recall": 0.8710199556541022,
        "support": 451
    },
    "enfisema": {
        "f1-score": 0.900645752817464,
        "precision": 0.9002239048695739,
        "recall": 0.9010956175298808,
        "support": 502
    },
    "espessamento": {
        "f1-score": 0.7585924954035294,
        "precision": 0.7817011455263224,
        "recall": 0.7368644067796604,
        "support": 590
    },
    "favo_de_mel": {
        "f1-score": 0.8472794562088324,
        "precision": 0.8421027069985625,
        "recall": 0.8525471698113207,
        "support": 530
    },
    "macro avg": {
        "f1-score": 0.855486663803172,
        "precision": 0.8548232250631667,
        "recall": 0.8564897953418324,
        "support": 3258
    },
    "normal": {
        "f1-score": 0.8757189198588096,
        "precision": 0.8657138546652738,
        "recall": 0